# Binary serialization
take a list of values (coming from the kernel, which are send to the browser), calculate the sum (which is send back).

In [1]:
import ipywidgets as widgets
import traitlets
class SumListWidget(widgets.DOMWidget):
    _model_name = traitlets.Unicode('SumModel').tag(sync=True)
    _model_module = traitlets.Unicode('sum').tag(sync=True)
    data = traitlets.List(traitlets.CFloat(), default_value=[1]).tag(sync=True)
    value = traitlets.CFloat()                                  .tag(sync=True, readonly=True)

In [4]:
%%javascript
require.undef('sum');

define('sum', ["@jupyter-widgets/base"], function(widgets) {

    var SumModel = widgets.WidgetModel.extend({
        defaults: function() {
            return _.extend(widgets.WidgetModel.prototype.defaults(), {
                _model_name : 'SumModel',
                _model_module : 'sum',
             })
        },
        initialize: function(attributes, options) {
            console.log(this)
            SumModel.__super__.initialize.apply(this, arguments);
            this.on('change:data', this.sum_data, this)
            this.sum_data()
        },
        sum_data: function() {
            var data = this.get('data')
            var total = 0;
            for(var i = 0; i < data.length; i++) {
                total += data[i]
            }
            this.set('value', total)
            this.save_changes()
        }
    });
    
    return {
        SumModel: SumModel
    }
});

<IPython.core.display.Javascript object>

In [3]:
sum_widget = SumListWidget(data=[1,2,3,4])

In [ ]:
sum_widget.value

In [ ]:
sum_widget.data = list(range(int(1e6)))
sum_widget.value

In [ ]:
sum_widget.value

In [6]:
import numpy as np

In [ ]:
sum_widget.data = np.arange(1e6)

In [7]:
class SumListWidget(widgets.DOMWidget):
    _model_name = traitlets.Unicode('SumModel').tag(sync=True)
    _model_module = traitlets.Unicode('sum').tag(sync=True)
    data = traitlets.Instance(np.ndarray).tag(sync=True)
    value = traitlets.CFloat()           .tag(sync=True, readonly=True)

In [8]:
sum_widget = SumListWidget(data=np.arange(1e6))

ValueError: Can't clean for JSON: array([  0.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
         9.99997000e+05,   9.99998000e+05,   9.99999000e+05])

In [9]:
def ndarray_to_json(ar, obj):
    return ar.tolist()
class SumListWidget(widgets.DOMWidget):
    _model_name = traitlets.Unicode('SumModel').tag(sync=True)
    _model_module = traitlets.Unicode('sum').tag(sync=True)
    data = traitlets.Instance(np.ndarray).tag(sync=True, to_json=ndarray_to_json)
    value = traitlets.CFloat()           .tag(sync=True, readonly=True)

In [10]:
sum_widget = SumListWidget(data=np.arange(1e3))

## Binary serialization
Allow sending of binary buffers / memoryviews over a websocket.

In [11]:
def ndarray_to_json(ar, obj):
    json_plus = {'buffer':memoryview(ar), 'dtype':str(ar.dtype), 'shape':ar.shape}
    print(json_plus)
    return json_plus


class SumListWidget(widgets.DOMWidget):
    _model_name = traitlets.Unicode('SumModel').tag(sync=True)
    _model_module = traitlets.Unicode('sum').tag(sync=True)
    data = traitlets.Instance(np.ndarray).tag(sync=True, to_json=ndarray_to_json)
    value = traitlets.CFloat()           .tag(sync=True, readonly=True)

In [12]:
sum_widget = SumListWidget(data=np.arange(1e6))

{'buffer': <memory at 0x111cac408>, 'dtype': 'float64', 'shape': (1000000,)}


In [13]:
previous_send = sum_widget._send
def peek_send(*args, **kwargs):
    print('what gets send is:')
    print('  args: ', args)
    print('  kwargs: ', kwargs)
    previous_send(*args, **kwargs)
sum_widget._send = peek_send
sum_widget.data = np.arange(2e6)
sum_widget._send = previous_send

{'buffer': <memory at 0x111cac408>, 'dtype': 'float64', 'shape': (2000000,)}
what gets send is:
  args:  ({'method': 'update', 'state': {'data': {'dtype': 'float64', 'shape': (2000000,)}}, 'buffer_paths': [['data', 'buffer']]},)
  kwargs:  {'buffers': [<memory at 0x111cac408>]}


## So we cannot send ndarrays, but why buffer/memoryview?
buffers / memoryviews get removed from the json, it's 'path' kept in 'buffer_paths', and refers to a list of buffers. On the frontend (browser) the json gets deserialized, and the buffers get put back into place.

In [14]:
%%javascript
require.undef('sum');

define('sum', ["@jupyter-widgets/base"], function(widgets) {

    function deserialize_numpy_array(json_plus, manager) {
        console.log("binary array", json_plus)
        var ar = new Float64Array(json_plus.buffer.buffer)
        return ar
    }

    var SumModel = widgets.WidgetModel.extend({
        defaults: function() {
            return _.extend(widgets.WidgetModel.prototype.defaults(), {
                _model_name : 'SumModel',
                _model_module : 'sum',
             })
        },
        initialize: function(attributes, options) {
            console.log(this)
            SumModel.__super__.initialize.apply(this, arguments);
            this.on('change:data', this.sum_data, this)
            this.sum_data()
        },
        sum_data: function() {
            var data = this.get('data')
            var total = 0;
            for(var i = 0; i < data.length; i++) {
                total += data[i]
                //console.log('x[', i, '] = ', x.data[i], '; y[', i, '] = ', y[i])
            }
            console.log('total', total)
            this.set('value', total)
            this.save_changes()
        }}, {
        serializers: _.extend({
            data: { deserialize: deserialize_numpy_array},
        }, widgets.WidgetModel.serializers)

    });
    
    return {
        SumModel: SumModel
    }
});

<IPython.core.display.Javascript object>

In [18]:
sum_widget = SumListWidget(data=np.arange(1e6))

{'buffer': <memory at 0x111cac348>, 'dtype': 'float64', 'shape': (1000000,)}


In [17]:
sum_widget.value

499999500000.0

# Standard (de)serialisers?
 * https://github.com/vidartf/ipydatawidgets
 * js ndarrays: https://github.com/scijs/ndarray